In [197]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
from lib import gauss_newton, levenberg_marquardt
import numpy as np
from typing import Callable

## Example functions

In [199]:
xs = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
ys = np.array([1, 2, 3, 4])


def f(x: np.ndarray, p: np.ndarray) -> np.ndarray:
    assert x.shape == (2,)
    assert p.shape == (3,)

    return p[0] * x[0] + p[1] * x[1] + p[2]


def df(x: np.ndarray, p: np.ndarray) -> np.ndarray:
    assert x.shape == (2,)
    assert p.shape == (3,)

    return np.array([x[0], x[1], 1])


def residue(f: Callable, xs: np.ndarray, ys: np.ndarray, p: np.ndarray) -> np.ndarray:
    assert xs.shape[1] == 2
    assert xs.shape[0] == ys.shape[0]
    assert p.shape == (3,)

    fs = np.array([f(x, p) for x in xs])

    return (fs - ys) ** 2


def residue_jacobian(
    f: Callable, df: Callable, ys: np.ndarray, xs: np.ndarray, p: np.ndarray
) -> np.ndarray:
    assert xs.shape[1] == 2
    assert xs.shape[0] == ys.shape[0]
    assert p.shape == (3,)

    fs = np.array([f(x, p) for x in xs])
    dfs = np.array([df(x, p) for x in xs])

    return np.array([2 * (fs[i] - ys[i]) * dfs[i] for i in range(len(xs))])


p0 = np.random.randn(3)
alpha = 0.01
max_iter = 1000

In [200]:
residue(f, xs, ys, p0)

array([1.25599626, 1.95493471, 2.8078606 , 3.81477392])

In [201]:
residue_jacobian(f, df, ys, xs, p0)

array([[ -2.24142477,  -4.48284955,  -2.24142477],
       [ -5.5927592 ,  -8.3891388 ,  -2.7963796 ],
       [-10.05400327, -13.40533769,  -3.35133442],
       [-15.62515698, -19.53144623,  -3.90628925]])

In [202]:
def F(p: np.ndarray) -> np.ndarray:
    return residue(f, xs, ys, p)


def DF(p: np.ndarray) -> np.ndarray:
    return residue_jacobian(f, df, ys, xs, p)

## Gauss-Newton method

In [203]:
p, err = gauss_newton(F, DF, p0, max_iter, silent=True)
p, err

(array([ 0.54021091,  0.45978697, -0.45979341]), 3.0710197257260987e-10)

In [204]:
np.array([f(xs[i], p) for i in range(len(xs))])

array([0.99999145, 1.99998933, 2.99998722, 3.9999851 ])

## Levenberg-Marquardt method

#### 1. Sequence $\lambda_k = const$

In [205]:
def lambda_fun(F: Callable, DF: Callable, p: np.ndarray, i: int) -> float:
    return 1e-3

In [206]:
p, err = levenberg_marquardt(lambda_fun, F, DF, p0, max_iter, silent=True)
p, err

(array([ 0.1304331 ,  0.86995918, -0.87102769]), 5.283007648373984e-07)

In [207]:
np.array([f(xs[i], p) for i in range(len(xs))])

array([0.99932376, 1.99971603, 3.00010831, 4.00050058])

#### 2. Sequence $\lambda_k = \lambda_0 \cdot \beta^k$